# Question 1: Scrape the Wikipedia page and preprocessing the data

## Scrape the Wikipedia page

In [168]:
#Import pandas packages to scrpe the webpage table
import requests
import lxml.html as lh
import pandas as pd
import requests
import math
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import matplotlib.cm as cm  # Matplotlib and associated plotting modules
import matplotlib.colors as colors
from matplotlib import pyplot as plt
from sklearn import metrics 
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans # KMeans for clustering
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

In [88]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [89]:
#let’s parse the first row as our header
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))
col[0:5]

1:"Postcode"
2:"Borough"
3:"Neighborhood
"


[('Postcode', []), ('Borough', []), ('Neighborhood\n', [])]

In [94]:
#Since our first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
[len(C) for (title,C) in col]

[287, 287, 287]

In [96]:
#Creat the dataframe
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.rename(columns={'Neighborhood\n': 'Neighborhood'}, inplace=True)
df['Neighborhood'] = df['Neighborhood'].str[:-1]
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [97]:
#The required dataframe has been created at this point. More processing would be done in section 1.2.
df.shape

(287, 3)

## Processing the cells in the dataframe to satisfy the requirements of the assignment.

In [113]:
#Remove the cells with a 'borough' value that is 'Not assigned'.
searchfor = ['Not assigned']
df = df[~df['Borough'].str.contains('|'.join(searchfor))]
del df['index']
del df['level_0']
df.head()
df.shape #77 rows has been removed due to their value of 'Borough' is 'Not assigned'

(210, 3)

In [142]:
#Combine cells in 'Neighborhood' column that share the same value in 'Postcode' column
TorontoNeighborhood = df
result = TorontoNeighborhood.groupby('Postcode')['Neighborhood'].agg(', '.join).reset_index()
result['Borough']=TorontoNeighborhood['Borough']
result['temp'] = result['Borough']
result['Borough'] = result['Neighborhood']
result['Neighborhood'] = result['temp']
result.drop(columns=['temp'], inplace=True)

result.rename(columns={'Borough': 'temp'}, inplace=True)
result.rename(columns={'Neighborhood': 'Borough'}, inplace=True)
result.rename(columns={'temp': 'Neighborhood'}, inplace=True)

result.shape

(103, 3)

In [152]:
#The dataframe 'refined_result' has replaced 'Not assigned' values in 'Neighborhood' column with the value in 'Borough' column
refined_result = result
refined_result.loc[(refined_result.Neighborhood == 'Not assigned'),'Neighborhood']= refined_result['Borough']
refined_result.shape

(103, 3)

# Question 2: Get the latitude and the longitude coordinates of each neighborhood and combine the dataframes as required

In [173]:
import json # library to handle JSON files
#!pip install wget
import wget
#Read the csv file that has the geographical coordinates of each postal code
!wget -q -O 'toronto_data.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')
toronto_postal = pd.read_csv('toronto_data.csv')

Data downloaded!


In [176]:
toronto_postal.rename(columns={'Postal Code':'Postcode'}, inplace=True)
toronto_postal.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [177]:
refined_result.head()

,Postcode,Borough,Neighborhood
0,M1B,North York,"Rouge, Malvern"
1,M1C,North York,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Downtown Toronto,"Guildwood, Morningside, West Hill"
3,M1G,North York,Woburn
4,M1H,North York,Cedarbrae


In [181]:
combined_toronto_data = pd.merge(refined_result, toronto_postal, how='inner', on = 'Postcode')
combined_toronto_data.shape

(103, 5)

In [183]:
combined_toronto_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,North York,"Rouge, Malvern",43.806686,-79.194353
1,M1C,North York,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Downtown Toronto,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,North York,Woburn,43.770992,-79.216917
4,M1H,North York,Cedarbrae,43.773136,-79.239476


# Question 3: Explore and cluster the neighborhoods in Toronto

## Get Toronto data from foursquare

In [194]:
!pip install folium
import folium
#Get my credencials on foursquare
CLIENT_ID = '1LFVRZ351MSJGMUVYZEYIMTIN2BY2V5AU5WRA1HWWWAMODJ4' # Foursquare ID
CLIENT_SECRET = 'FHJBRKSNHCP1QM32R5OMWSN0DIEDGCIHFVUBIXCVMKJOT5UG' # Foursquare Secret
VERSION = '20180604'
LIMIT = 50

Display the neighborhood of Toronto using the dataframe 'combined_toronto_data' generated in the last section

In [188]:
#Get the coordinates of Toronto
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="Foursquare_agent")
location = geolocator.geocode(address)
Toronto_latitude = location.latitude
Toronto_longitude = location.longitude
print('Toronto Geographical coordinates are Lat={}, Lng={} ' .format(Toronto_latitude, Toronto_longitude))

Toronto Geographical coordinates are Lat=43.653963, Lng=-79.387207 


In [193]:
#Display the map of Toronto with markers
Toronto_map = folium.Map(location=[Toronto_latitude, Toronto_longitude], zoom_start=11, control_scale = True)

for lat, lng, Borough, Neighbourhood in zip(combined_toronto_data['Latitude'], combined_toronto_data['Longitude'], combined_toronto_data['Borough'], combined_toronto_data['Neighborhood']):
    tag = '{}, {}'.format(Neighbourhood, Borough)
    tag = folium.Popup(tag, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=tag,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html=False).add_to(Toronto_map)

# adding a folium fetaure allowing location [Neighborhood, Borough], and coordinates shown with mouse-over and 
# mouse click on the map 
Toronto_map.add_child(folium.LatLngPopup())
    
Toronto_map


In [264]:
#Generate a new dataframe that 'Toronto' exists in 'Borough'. This would be the dataset that our exploring and clustering would be done.
Toronto_neighborhoods = combined_toronto_data
Toronto_neighborhoods = Toronto_neighborhoods[Toronto_neighborhoods['Borough'].str.contains('Toronto')]
Toronto_neighborhoods.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M1E,Downtown Toronto,"Guildwood, Morningside, West Hill",43.763573,-79.188711
6,M1K,Downtown Toronto,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
12,M1S,Downtown Toronto,Agincourt,43.794200,-79.262029
13,M1T,Downtown Toronto,"Clarks Corners, Sullivan, Tam O'Shanter",43.781638,-79.304302
26,M3B,Downtown Toronto,Don Mills North,43.745906,-79.352188


Define a function to get the venue info from the newly generated dataframe 'Toronto_neighborhoods'

In [265]:
def getNearbyVenues (names, latitude, longitude, radius=300, limit=100):
    
    venue_lst=[] # initialize empty list
    
    for name, lat, lng in zip (names, latitude, longitude):
        print(name)
        
    
        # Foursuare API explore generated per Neighbourhood lat / lng passed from toronto_df
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
        
        # Get request
        results = requests.get(url).json()["response"]['groups'][0]['items']
                
        # return only relevant information for each nearby venue in the preliminary list
        venue_lst.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    venue_df = pd.DataFrame([item for venue_lst in venue_lst for item in venue_lst])
    venue_df.columns = ['Neighbourhood', 
                  'Neigh Lat', 
                  'Neigh Lng', 
                  'Venue', 
                  'Venue Lat', 
                  'Venue Lng', 
                  'Venue Category']

    print('{} venues were returned by Foursquare.'.format(venue_df.shape[0]))
    
    return (venue_df)

In [266]:
Toronto_venus = getNearbyVenues (names=Toronto_neighborhoods['Neighbourhood'], 
                                     latitude=Toronto_neighborhoods['Latitude'], 
                                     longitude=Toronto_neighborhoods['Longitude'])

Guildwood, Morningside, West Hill
East Birchmount Park, Ionview, Kennedy Park
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Don Mills North
Woodbine Gardens, Parkview Hill
Woodbine Heights
East Toronto
The Danforth West, Riverdale
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Bedford Park, Lawrence Manor East
Roselawn
Forest Hill North, Forest Hill West
Lawrence Heights, Lawrence Manor
Glencairn
Humewood-Cedarvale
Caledonia-Fairbanks
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Runnymede, Swansea
East Toronto
Canada Post Gateway Processing Centre
Business Reply Mail Processing Centre 969 Eastern
Weston
419 venues were returned by Foursquare.


In [267]:
Toronto_venus.shape

(419, 7)

In [268]:
Toronto_venus.head()

,Neighbourhood,Neigh Lat,Neigh Lng,Venue,Venue Lat,Venue Lng,Venue Category
0,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
1,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,Corvette Park,43.725826,-79.259852,Playground
2,Agincourt,43.794200,-79.262029,Panagio's Breakfast & Lunch,43.792370,-79.260203,Breakfast Spot
3,Agincourt,43.794200,-79.262029,Subway,43.792823,-79.259681,Sandwich Place
4,Agincourt,43.794200,-79.262029,El Pulgarcito,43.792648,-79.259208,Latin American Restaurant


Check how many were returned for Toronto neighborhood

In [269]:
Toronto_venus.groupby('Neighbourhood').count()

,Neigh Lat,Neigh Lng,Venue,Venue Lat,Venue Lng,Venue Category
Neighbourhood,,,,,,
Agincourt,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",14,14,14,14,14,14
Business Reply Mail Processing Centre 969 Eastern,7,7,7,7,7,7
"Cabbagetown, St. James Town",21,21,21,21,21,21
Caledonia-Fairbanks,4,4,4,4,4,4
Canada Post Gateway Processing Centre,4,4,4,4,4,4
Christie,7,7,7,7,7,7
Church and Wellesley,52,52,52,52,52,52
"Clarks Corners, Sullivan, Tam O'Shanter",3,3,3,3,3,3


Let's find out how many unique categories can be curated from all the returned venues

In [270]:
print('There are {} uniques categories.'.format(len(Toronto_venus['Venue Category'].unique())))

There are 133 uniques categories.


Now, let's analyze the neighborhoods. We need to ensure the Categorical values for the Venue Category are encoded to numerical - we use 'pd.get_dummies'

In [271]:
# one hot encoding
Toronto_neighborhoods_onehot = pd.get_dummies(Toronto_venus[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_neighborhoods_onehot['Neighbourhood'] = Toronto_venus['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_neighborhoods_onehot.columns[-1]] + list(Toronto_neighborhoods_onehot.columns[:-1])
Toronto_neighborhoods_onehot = Toronto_neighborhoods_onehot[fixed_columns]

Toronto_neighborhoods_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,American Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Workshop,Bakery,Bank,...,Thai Restaurant,Theater,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"East Birchmount Park, Ionview, Kennedy Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Agincourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Agincourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Agincourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size

In [272]:
Toronto_neighborhoods_onehot.shape

(419, 134)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [273]:
Toronto_grouped = Toronto_neighborhoods_onehot.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped

,Neighbourhood,Accessories Store,Adult Boutique,American Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Workshop,Bakery,Bank,...,Thai Restaurant,Theater,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.142857,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Cabbagetown, St. James Town",0.000000,0.000000,0.047619,0.000000,0.000000,0.0,0.000000,0.047619,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Caledonia-Fairbanks,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000
5,Canada Post Gateway Processing Centre,0.000000,0.000000,0.250000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Christie,0.000000,0.000000,0.142857,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Church and Wellesley,0.000000,0.019231,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.019231,0.000000,0.019231,0.000000,0.019231,0.000000,0.019231,0.019231,0.000000,0.000000
8,"Clarks Corners, Sullivan, Tam O'Shanter",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.333333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.025641,0.012821,0.012821,0.0,0.000000,0.025641,0.000000,...,0.012821,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000,0.000000,0.000000,0.000000


Let's confirm the new size

In [274]:
Toronto_grouped.shape

(30, 134)

Let's print each neighborhood along with the top 5 most common venues

In [275]:
num_top_venues = 5

for hood in Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Sandwich Place  0.25
1             Breakfast Spot  0.25
2       Pakistani Restaurant  0.25
3  Latin American Restaurant  0.25
4                     Museum  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.14
1         Coffee Shop  0.14
2    Sushi Restaurant  0.14
3     Thai Restaurant  0.07
4          Restaurant  0.07


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0               Brewery  0.14
1  Fast Food Restaurant  0.14
2                  Park  0.14
3         Auto Workshop  0.14
4                Garden  0.14


----Cabbagetown, St. James Town----
               venue  freq
0               Café  0.10
1        Pizza Place  0.10
2        Coffee Shop  0.10
3              Diner  0.05
4  Indian Restaurant  0.05


----Caledonia-Fairbanks----
                  venue  freq
0                Market  0.25
1         Women's Store  0.25


Let's put that into a pandas dataframe
First, let's write a function to sort the venues in descending order.

In [276]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [277]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Latin American Restaurant,Pakistani Restaurant,Sandwich Place,Electronics Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Dog Run
1,"Bedford Park, Lawrence Manor East",Sushi Restaurant,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Pub,Restaurant,Sandwich Place,Liquor Store,Juice Bar,Comfort Food Restaurant
2,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Auto Workshop,Fast Food Restaurant,Farmers Market,Brewery,Garden,Park,Dessert Shop,Deli / Bodega,Diner
3,"Cabbagetown, St. James Town",Coffee Shop,Pizza Place,Café,Sandwich Place,Gym / Fitness Center,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Liquor Store
4,Caledonia-Fairbanks,Fast Food Restaurant,Women's Store,Park,Market,Dog Run,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Yoga Studio


## Clustering neighborhoods!

In [278]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighbourhood', 1)
#Toronto_grouped_clustering.head()
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:30]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1,
       3, 3, 2, 1, 1, 3, 1, 0], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [281]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

Toronto_merged = Toronto_neighborhoods
Toronto_merged.rename({'Neighborhood':'Neighbourhood'}, inplace=True)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Toronto_merged.head(20) # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M1E,Downtown Toronto,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4,Electronics Store,Yoga Studio,Fish & Chips Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Diner
6,M1K,Downtown Toronto,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,1,Playground,Fish & Chips Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Dog Run
12,M1S,Downtown Toronto,Agincourt,43.794200,-79.262029,1,Breakfast Spot,Latin American Restaurant,Pakistani Restaurant,Sandwich Place,Electronics Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Dog Run
13,M1T,Downtown Toronto,"Clarks Corners, Sullivan, Tam O'Shanter",43.781638,-79.304302,1,Bank,Shopping Mall,Fast Food Restaurant,Yoga Studio,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Dog Run,Fish & Chips Shop
26,M3B,Downtown Toronto,Don Mills North,43.745906,-79.352188,1,Tennis Court,Pool,Basketball Court,Baseball Field,Yoga Studio,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store
35,M4B,East Toronto,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,1,Pizza Place,Bank,Café,Pet Store,Breakfast Spot,Intersection,Pharmacy,Gastropub,Garden,Falafel Restaurant
36,M4C,Downtown Toronto,Woodbine Heights,43.695344,-79.318389,0,Beer Store,Yoga Studio,Electronics Store,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Dog Run,Flower Shop
40,M4J,Downtown Toronto,East Toronto,43.685347,-79.338106,1,Coffee Shop,Park,Italian Restaurant,Bubble Tea Shop,Bakery,Café,Sushi Restaurant,Middle Eastern Restaurant,Portuguese Restaurant,Sandwich Place
41,M4K,Downtown Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Ice Cream Shop,Restaurant,Yoga Studio,Bookstore,Cosmetics Shop,Coffee Shop,Dessert Shop,Diner,Pub
48,M4T,Downtown Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,3,Intersection,Park,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Dog Run,Yoga Studio


Let's visiualize the clusters!

In [282]:
# create map
map_clusters = folium.Map(location=[Toronto_latitude, Toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine the clusters

In [284]:
Toronto_merged.loc[Toronto_merged['Cluster_Labels'] == 0, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,Woodbine Heights,0,Beer Store,Yoga Studio,Electronics Store,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Dog Run,Flower Shop


In [285]:
Toronto_merged.loc[Toronto_merged['Cluster_Labels'] == 1, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,"East Birchmount Park, Ionview, Kennedy Park",1,Playground,Fish & Chips Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Dog Run
12,Agincourt,1,Breakfast Spot,Latin American Restaurant,Pakistani Restaurant,Sandwich Place,Electronics Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Dog Run
13,"Clarks Corners, Sullivan, Tam O'Shanter",1,Bank,Shopping Mall,Fast Food Restaurant,Yoga Studio,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Dog Run,Fish & Chips Shop
26,Don Mills North,1,Tennis Court,Pool,Basketball Court,Baseball Field,Yoga Studio,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store
35,"Woodbine Gardens, Parkview Hill",1,Pizza Place,Bank,Café,Pet Store,Breakfast Spot,Intersection,Pharmacy,Gastropub,Garden,Falafel Restaurant
40,East Toronto,1,Coffee Shop,Park,Italian Restaurant,Bubble Tea Shop,Bakery,Café,Sushi Restaurant,Middle Eastern Restaurant,Portuguese Restaurant,Sandwich Place
41,"The Danforth West, Riverdale",1,Greek Restaurant,Ice Cream Shop,Restaurant,Yoga Studio,Bookstore,Cosmetics Shop,Coffee Shop,Dessert Shop,Diner,Pub
49,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",1,Coffee Shop,Supermarket,Athletics & Sports,Liquor Store,Yoga Studio,Electronics Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant
51,"Cabbagetown, St. James Town",1,Coffee Shop,Pizza Place,Café,Sandwich Place,Gym / Fitness Center,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Liquor Store
52,Church and Wellesley,1,Gay Bar,Burger Joint,Coffee Shop,Japanese Restaurant,Café,Italian Restaurant,Bubble Tea Shop,Pizza Place,Park,Middle Eastern Restaurant


In [286]:
Toronto_merged.loc[Toronto_merged['Cluster_Labels'] == 2, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Roselawn,2,Health & Beauty Service,Fish & Chips Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Dog Run


In [287]:
Toronto_merged.loc[Toronto_merged['Cluster_Labels'] == 3, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,"Moore Park, Summerhill East",3,Intersection,Park,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Dog Run,Yoga Studio
50,Rosedale,3,Park,Building,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Diner
98,Weston,3,Park,Yoga Studio,Fish & Chips Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Dog Run


In [288]:
Toronto_merged.loc[Toronto_merged['Cluster_Labels'] == 4, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Guildwood, Morningside, West Hill",4,Electronics Store,Yoga Studio,Fish & Chips Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Diner
